In [ ]:
%pip install earthengine-api -q
%pip install folium -q

In [ ]:
import ee
import folium
import pandas as pd

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:

CLASS_NAMES = [
    "water",
    "trees",
    "grass",
    "flooded_vegetation",
    "crops",
    "shrub_and_scrub",
    "built",
    "bare",
    "snow_and_ice",
]

VIS_PALETTE = [
    "419BDF",
    "397D49",
    "88B053",
    "7A87C6",
    "E49635",
    "DFC35A",
    "C4281B",
    "A59B8F",
    "B39FE1",
]
position = (26, 113) # China

In [ ]:
dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
dwImage = ee.Image(dw.filterDate("2022-10-01", "2022-10-07").median())
dwRgb = dwImage.select('label').visualize(min=0, max=8, palette=VIS_PALETTE).divide(255)
top1Prob = dwImage.select(CLASS_NAMES).reduce(ee.Reducer.max())
top1ProbHillshade = ee.Terrain.hillshade(top1Prob.multiply(100)).divide(255)
dwRgbHillshade = dwRgb.multiply(top1ProbHillshade)

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name, opacity=0.5):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
        opacity=opacity,
    ).add_to(self)


folium.Map.add_ee_layer = add_ee_layer


def display_map(position, image: ee.Image, zoom=10):
    my_map = folium.Map(location=position, zoom_start=zoom)
    my_map.add_ee_layer(image, {"min": 0, "max": 0.65}, "Dynamic World", opacity=0.4)
    display(my_map)


In [ ]:
display_map(position, dwRgbHillshade)